# Machine Learning with Spark ML

### In this notebook, we will explore machine learning using Spark ML. We will exploit Spark ML's high-level APIs built on top of DataFrames to create and tune machine learning pipelines. Spark ML Pipelines enable combining multiple algorithms into a single pipeline or workflow. We will utilize Spark ML's feature transformers to convert, modify and scale the features that will be used to develop the machine learning model. Finally, we will evaluate and cross validate our model to demonstrate the process of determining a best fit model and load the results in the database.

### We are using machine learning to try to predict diagnoses of breast cancer tumors. We will use data on tumors that have already been diagnosed as benign or malignant as a training set for the algorithm .¶

## Table of Contents 
1. [Create Version](#version)
2. [Import Libraries](#import)
3. [Read from Cloud Object Storage](#read)
4. [Transform data](#transform)
5. [PixieDust](#pixie)
6. [Feature Engineering](#engineer)
7. [Define model](#model)
8. [Create pipeline](#pipeline)
9. [Train model](#train)
10. [Evaluate accuracy](#evaluate)
11. [Hyperparameter Tuning](#tuning)
12. [Choose and Utilize the best-fit model](#best)
13. [Save Results to Cloud Object Storage](#credentials)

<a id="version"></a>
## Create Version 
Save a version of the notebook by selecting <b>File</b> > <b>Save Version</b> 
<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/FileOptions.PNG" > or by selecting the <b>Versions</b> icon and selecting <b>Save Version</b>. <img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/jpatter/LMCO/master/Lab-1/images/versions-button.png" ><br>
You can have up to ten (10) versions of a notebook.   Notebook versions are saved in a FIFO manner.

## Ensure Spark is installed and up to date 

In [ ]:
print('The spark version is {}.'.format(spark.version))

<a id="import"></a>
## Import the required libraries 

In [ ]:
#Imports for Spark
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.feature import Bucketizer
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import NaiveBayes, DecisionTreeClassifier
from pyspark.sql.functions import year
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Imports for pixiedust
from pixiedust.display import *

<a id="read"></a>
## Read from Cloud Object Storage
Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, which is the <b>Find and Add Data</b> icon.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/bleonardb3/WatsonStudio/master/Lab-1/images/onezeroicon.png" >

Select the Files view then click on **Insert to code**, then click on **Insert SparkSession DataFrame**.

Rename the dataframe to **cancer_df**.

In [ ]:
# Insert SparkSession DataFrame here
# make CERTAIN to rename the default dataframe name to cancer_df
import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': '54eK5hQft9uVYbrpsxfWs4Tdgw8sbc8weZndKdj8iwhj',
    'service_id': 'iam-ServiceId-e54eecd9-137d-44ca-b6d3-87b14b0ad21a',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_af2c2a9305dd4cc6a1ce91fbe599e4bb_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
cancer_df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('cancer_data.csv', 'mlproject-donotdelete-pr-rwqegunlryj5yq'))
cancer_df.take(5)


<a id="transform"></a>
## Identify labels and features to transform
A label is the output of an ML model. Since we are creating a model to predict whether a tumor is malignant or benign our label will be **diagnosis**.  

A feature is an input of the ML model, in this case, the columns that we are using to help predict the diagnosis of a tumor (area, smoothness, etc). We will need to tranform each feature into a double so that it can be used properly later on to create, train and test the model.

Spark uses Resilient Distributed Datasets (RDDs) which are immutable, meaning if we need to transform a column of the dataframe we must create a new RDD each time we perform a transformation. We can do so using withColumn() and because each feature must be cast as a double we need to transform each feature column.

In [ ]:
df_cols = ["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst"]

dfprev = (cancer_df.withColumn("idTemp", cancer_df["id"]
    .cast("Double")).drop("id").withColumnRenamed("idTemp", "id"))

for df_col in df_cols:
    dfnew = (dfprev.withColumn("temp", dfprev[df_col]
             .cast("Double")).drop(df_col).withColumnRenamed("temp",df_col))
    dfprev = dfnew
    
cancer_nums = dfprev

<a id="pixie"></a>
## Using PixieDust 
We can use PixieDust to explore the data. PixieDust is an open source tool to add visualizations to Jupyter Notebooks and simplifies the visual options of python.

By clicking the grid icon in the output of the cell below we can see a table of the dataframe.

If we click the drop down menu (next to the grid icon), symbolized with the graph, we can choose to put the data into a scatterplot. We can choose **id** as the key, one of the feature columns (**area_mean**, **perimeter_mean**, etc.) as the value and set the number of rows to display to 1000. We are given a scatterplot of our data points and by choosing the bokeh renderer on the right side and setting the color to diagnosis we can see the spread of the different diagnoses.

Example: Set the key for the graph to **id**, the value to **area_mean**, and color to diagnosis. The plot provided allows us to infer that most benign tumors have smaller mean area when compared to malignant tumors. On the far right of the graph we are presented with our unlabeled points (green points) and using the trend of the labeled points we can predict which of the unlabeled tumors may be benign and malignant.

In [ ]:
display(cancer_nums)

We can also use PixieDust to observe the composition of the data. 
Below we group the data by diagnosis so that we can get a count of the number of each diagnosis (M-malignant, B-benign, U-unlabeled). 

Using the table feature we can see that the data has 305 benign tumors, 193 malignant tumors, and 71 unlabeled tumors. 

We can also use a pie chart, created using the drop down menu used to make the scatterplot above, to see the percentages of each diagnosis.

In [ ]:
cancer = cancer_nums.groupby("diagnosis").count()
display(cancer)

Because some of our data points are unlabeled we need to filter them out before we train our model.

In [ ]:
LabeledData = cancer_nums.filter("diagnosis != 'U'")

<a id="engineer"></a>
## Feature Engineering 
We need to change our label, **diagnosis**, into a form that SparkML can use. We can do so by using StringIndexer which converts textual data into numeric data while keeping the context categorical. It encodes the input column, in our case the diagnosis column, to a column of indices between 0 and the number of labels where the most frequent label has index 0.

To turn the label back into a human readable form we can use the converter that we define below using IndexToString. IndexToString maps the column of indices back to the original labels of the column.

<a id="engineer"></a>
## Feature Engineering 
We need to change our label, **diagnosis**, into a form that SparkML can use. We can do so by using StringIndexer which encodes the input column, in our case the diagnosis column, to a column of indices that are representative of the frequency of the input string.

To turn the label back into a human readable form we can use the converter that we define below using IndexToString. IndexToString maps the column of indices back to the original labels of the column.

In [ ]:
labelIndexer = StringIndexer(inputCol="diagnosis", outputCol="label", handleInvalid="error")
labelModel = labelIndexer.fit(LabeledData)
converter = IndexToString(inputCol="prediction", outputCol="predCategory", labels=labelModel.labels)

Now we must put all feature columns into an array using VectorAssembler which combines a list of columns into a single vector column that can be used in training the model.

In [ ]:
vecAssembler = VectorAssembler(inputCols=["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst"], outputCol="features")

We use a normalizer to normalize each vector to have a standard form to improve the algorithm.

In [ ]:
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=2.0)

<a id="model"></a>
## State the model to be used 
There are many different types of models that can be used with Spark from Naïve Bayes to Random Forest. For our dataset a Logistic Regression model is most appropriate since they are often used for models of binary categorical outcome. The model will output its prediction into the **prediction** column.

For more information on the models that can be used with Spark and how to determine which model to use for your dataset see: https://spark.apache.org/docs/2.2.0/ml-classification-regression.html

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

<a id="pipeline"></a>
## Create a pipeline
A pipeline is a sequence of stages that are to be run in a specific order where each stage is either a transformer or an estimator. Transformers convert dataframes into other dataframes, usually by appending a column, while estimators, such as LogisticRegression, call fit() and train a Logistic Regression Model which is a transformer since it adds a **predictions** column. A pipeline chains together multiple tranformers to specify a specific ML workflow. 

In our case, we first want to tranform the label into a usable form for the algorithm, then we create a combined vector, normalize the vector, fit the model, and finally convert the results back into the original label format.

For more information on Pipelines visit https://spark.apache.org/docs/latest/ml-pipeline.html#pipeline-components

In [ ]:
pipeline = Pipeline(stages=[labelIndexer, vecAssembler, normalizer, lr, converter])

<a id="train"></a>
## Divide data into a training and testing set 
We should split the data randomly with 70% going to the training set and 30% going to the testing set.

In [ ]:
train, test = LabeledData.randomSplit([70.0,30.0], seed=1)
train.cache()
test.cache()
print('The number of records in the training data set is {}.'.format(train.count()))
print('The number of rows labeled M is {}.'.format(train.filter(train['diagnosis'] == "M").count()))
print('The number of rows labeled B is {}.'.format(train.filter(train['diagnosis'] == "B").count()))

print()

print('The number of records in the test data set is {}.'.format(test.count()))
print('The number of rows labeled M is {}.'.format(test.filter(train['diagnosis'] == "M").count()))
print('The number of rows labeled B is {}.'.format(test.filter(train['diagnosis'] == "B").count()))

## Fit the model and make predictions 
Next, we need to fit the pipeline to the model and make predictions using the model.

In [ ]:
model = pipeline.fit(train)
predictions = model.transform(test)
predictions.count()

<a id="evaluate"></a>
## Evaluate the accuracy of the model 
We can use the Receiver Operator Characteristic (ROC) curve for binary classifiers which assess the models diagnositc ability. The ROC is useful for comparing models and choosing the best one for the data. An ROC value close to 1 performs very well, whereas a model with an ROC value close to 0.5 is as good as flipping a coin. The value is calculated by plotting the true positive rate (recall / probability of detection) against the false positive rate (fall-out / probability of a false alarm) at various levels.

In [ ]:
evaluator = BinaryClassificationEvaluator().setLabelCol("label").setMetricName("areaUnderROC")
print('Area under the ROC curve = {}.'.format(evaluator.evaluate(predictions)))

<a id="tuning"></a>
## Hyperparameter Tuning
To find the best possible model we can use model selection tools that:

1. Split the data into training and testing datasets
2. For each (training, test) pair, iterate through the set of ParamMaps
3. For each ParamMap, fit the pipeline using those parameters, get a fitted Model, and evaluate the Model’s performance
4. Select the Model found using the best-performing set of parameters

We must first specify which parameter values we would like the tool to test in order to find the best model. For our model we have the option to test different values for normalizer and/or any of the parameters of the Logistic Regression Model (maxIter, elasticNetParam, regParam, etc). 

One of the parameters being adjusted below is elasticNetParam, which must be between 0 and 1. If you would like your model to be closer to a Lasso regression and therefore set coefficients that are not relevant equal to 0, you can choose a value close to 1. If you'd like your model to be closer to a Ridge regression and therefore minimize the impact of irrelevant coefficients without setting them to 0 and regularize the model, you can choose a value close to 0.

The second parameter in the paramGrid below is the normalizer, which is important because it ensures that the algorithm runs correctly. The value that you set for the normalizer represents the p-norm used for normalization. 

To tune our model we will use cross-validation.
A CrossValidator splits the dataset into folds which are used as separate training and testing datasets. We will use 10 folds, so the CrossValidator will generate 10 dataset pairs for training and testing. It will use the average evaluation of the 10 models found using each dataset pair to identify the best ParamMaps and re-fit the pipeline using the best ParamMap for the entire dataset.

When working with Spark you also have the option to do hyperparameter tuning using a train-validation split which evaluates each ParamMap once, as opposed to k times like cross-validaition. The benefit of train-validation is that it is less expensive but it can be much less reliable if the training dataset isn't sufficiently large.

In [ ]:
paramGrid = (ParamGridBuilder().addGrid(lr.elasticNetParam, [0, 0.5, 1.0])
                 .addGrid(normalizer.p, [1.0, 3.0]).build())

In [ ]:
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(10)

In [ ]:
cvModel = cv.fit(train)
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))

Our cross-validated model was 0.58% better at predicting a diagnosis than our original model so we will use the new model for our final predictions. 

In [ ]:
print('Area under the ROC curve for non-tuned model = {}.'.format(evaluator.evaluate(predictions)))
print('Area under the ROC curve for best fitted model = {}.'.format(evaluator.evaluate(cvModel.transform(test))))
print('Improvement = {0:0.2f}%'.format((evaluator.evaluate(cvModel.transform(test)) - evaluator.evaluate(predictions)) *100 / evaluator.evaluate(predictions)))

<a id="best"></a>
## Finalizing and making predictions using the best model
First we must filter the data to have only the points that are unlabeled so that we can make predictions on diagnoses for those points.

In [ ]:
UnlabeledData=cancer_nums.filter("diagnosis == 'U'")

In [ ]:
newPreds = cvModel.transform(UnlabeledData)

In [ ]:
newPreds.select("id","prediction","predCategory","radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean","radius_se","texture_se","perimeter_se","area_se","smoothness_se","compactness_se","concavity_se","concave points_se","symmetry_se","fractal_dimension_se","radius_worst","texture_worst","perimeter_worst","area_worst","smoothness_worst","compactness_worst","concavity_worst","concave points_worst","symmetry_worst","fractal_dimension_worst").show()

Below we can see the overall prediction count for diagnoses

In [ ]:
print('The number of records in the unlabeled data set is {}.'.format(newPreds.count()))
print('The number of rows labeled malignant is {}.'.format(newPreds.filter(newPreds['predCategory'] == "M").count()))
print('The number of rows labeled benign is {}.'.format(newPreds.filter(newPreds['predCategory'] == "B").count()))

<a id="credentials"></a>
## Insert the object storage credentials

Click on the cell below, then on the notebook toolbar, click the box of 1's and 0's, which is the <b>Find and Add Data</b> icon.

<img alt="IBM Bluemix.Get started now" src="https://raw.githubusercontent.com/bleonardb3/WatsonStudio/master/Lab-1/images/onezeroicon.png" >

Under the cancer_data.csv file, click <b>Insert to code</b> then select the <b>Insert Credentials</b> link to have a credentials dictionary added to the notebook.  



Connecting to Cloud Object Storage requires the following information which are provided by the credentials dictionary inserted. 

    IBM_API_KEY
    IAM_SERVICE_ID
    ENDPOINT
    IBM_AUTH_ENDPOINT

We will upload the results file to the same bucket that contains the cancer_data.csv file.  
   
The @hidden_cell directive tells DSX not to export credentials when sharing.

In [ ]:
# Insert object storage credentials below
# Make sure the name that is used is credentials. If credentials_1 is shown, please change to credentials. 
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    'IBM_API_KEY_ID': '54eK5hQft9uVYbrpsxfWs4Tdgw8sbc8weZndKdj8iwhj',
    'IAM_SERVICE_ID': 'iam-ServiceId-e54eecd9-137d-44ca-b6d3-87b14b0ad21a',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'mlproject-donotdelete-pr-rwqegunlryj5yq',
    'FILE': 'cancer_data.csv'
}


In [ ]:
valuesToWrite= newPreds.select("id",  "predCategory","diagnosis").toPandas().to_csv(path_or_buf="cancer_results.csv",index=False)

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config
service_endpoint=credentials['ENDPOINT']
auth_endpoint=credentials['IBM_AUTH_ENDPOINT']
cos = ibm_boto3.client('s3',
                      ibm_api_key_id=credentials['IBM_API_KEY_ID'],
                      ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

In [ ]:
file="cancer_results.csv"
bucket = credentials['BUCKET']
cos.upload_file(Filename=file, Bucket=bucket, Key=file)

Now we can look in our Cloud Object Storage on the IBM Cloud for the results of our predictions. Within the bucket specified for this project (found in credentials inserted 4 cells above), we can find a .csv file that contains the results of our models prediction and we are able to state whether a tumor is benign or malignant with confidence.